In [ ]:
import requests
from datetime import datetime

tickers = ['IBM', 'MSFT', 'GOOGL', 'ORCL', 'JNJ', 'PFE', 'MRK', 'UNH', 'CAT', 'VMC', 'FLR', 'ACM', 'JPM', 'BAC', 'GS', 'WFC', 'TSLA', 'F', 'GM', 'TM']
API_KEY = 'RNGDZP0O02PRIQQF'
usd_to_eur = 0.92
years = [str(y) for y in range(2017, 2025)]

def fetch_overview(symbol):
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={API_KEY}'
    response = requests.get(url)
    return response.json() if response.status_code == 200 else {}

def fetch_stock_price_quarter_avg(symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol={symbol}&apikey={API_KEY}'
    response = requests.get(url)
    if response.status_code != 200:
        return None
    data = response.json()
    time_series = data.get("Monthly Adjusted Time Series", {})
    if not time_series:
        return None

    dates = sorted(time_series.keys(), reverse=True)
    months = {'03', '06', '09', '12'}
    quarterly_prices = []

    for date in dates:
        month = date.split("-")[1]
        if month in months:
            close_price = float(time_series[date]["4. close"])
            quarterly_prices.append(close_price)
            if len(quarterly_prices) == 4:
                break
    if len(quarterly_prices) < 4:
        return None
    return [i * usd_to_eur for i in quarterly_prices ]

def extract_and_convert(data, stock_price_eur):
    try:
        revenue = float(data.get('RevenueTTM', 0)) * usd_to_eur / 1_000_000 / 4
        profit = float(data.get('NetIncomeTTM', 0)) * usd_to_eur / 1_000_000 / 4
        rd = float(data.get('ResearchAndDevelopmentExpenseTTM', 0)) * usd_to_eur / 1_000_000 / 4
        market_cap = float(data.get('MarketCapitalization', 0)) * usd_to_eur / 1_000_000_000
        operating_cost = revenue - profit if revenue and profit else None

        return {
            'Revenue(€M)': round(revenue, 2),
            'Operating_Cost(€M)': round(operating_cost, 2) if operating_cost else None,
            'Net_Profit(€M)': round(profit, 2),
            'R&D_Spending(€M)': round(rd, 2),
            'Stock_Price(€)': stock_price_eur,
            'Market_Cap(€B)': round(market_cap, 2)
        }
    except Exception as e:
        print(f"Error processing data: {e}")
        return None

results = {}

for ticker in tickers:
    overview_data = fetch_overview(ticker)
    stock_price = fetch_stock_price_quarter_avg(ticker)

    if overview_data and stock_price:
        metrics = extract_and_convert(overview_data, stock_price)
        if metrics:
            results[ticker] = metrics

for ticker, vals in results.items():
    print(f"{ticker}: {vals}")



Processing IBM...

Processing MSFT...

Processing GOOGL...

Processing ORCL...

==== IBM Financial Summary ====
2017: {'Stock_Price(€)': [141.15, 133.47, 141.52, 160.21], 'Revenue(€M)': 72807.88, 'Net_Profit(€M)': 5292.76, 'Operating_Cost(€M)': 67515.12, 'R&D_Spending(€M)': 5142.8}
2018: {'Stock_Price(€)': [104.58, 139.11, 128.52, 141.16], 'Revenue(€M)': 73223.72, 'Net_Profit(€M)': 8029.76, 'Operating_Cost(€M)': 65193.96, 'R&D_Spending(€M)': 4948.68}
2019: {'Stock_Price(€)': [123.32, 133.79, 126.87, 129.81], 'Revenue(€M)': 53096.88, 'Net_Profit(€M)': 8676.52, 'Operating_Cost(€M)': 44420.36, 'R&D_Spending(€M)': 5437.2}
2020: {'Stock_Price(€)': [115.81, 111.94, 111.11, 102.06], 'Revenue(€M)': 67731.32, 'Net_Profit(€M)': 5142.8, 'Operating_Cost(€M)': 62588.52, 'R&D_Spending(€M)': 5826.36}
2021: {'Stock_Price(€)': [122.97, 127.82, 134.86, 122.6], 'Revenue(€M)': 52762.92, 'Net_Profit(€M)': 5282.64, 'Operating_Cost(€M)': 47480.28, 'R&D_Spending(€M)': 5968.96}
2022: {'Stock_Price(€)': [129.6

In [ ]:
import requests
import csv
from collections import defaultdict

API_KEY = 'IL9NH1YGNRZN6M5O'
USD_TO_EUR = 0.92
TICKER = 'UNH'

YEARS = [str(y) for y in range(2017, 2025)]
QUARTERS = {'Q1': '03', 'Q2': '06', 'Q3': '09', 'Q4': '12'}

def fetch_monthly_prices(symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol={symbol}&apikey={API_KEY}'
    r = requests.get(url)
    data = r.json().get("Monthly Adjusted Time Series", {})
    prices = {}
    for date, values in data.items():
        year, month = date[:4], date[5:7]
        if year in YEARS and month in QUARTERS.values():
            prices[f"{year}-{month}"] = round(float(values["4. close"]) * USD_TO_EUR, 2)
    return prices

def fetch_income_statement(symbol):
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={API_KEY}'
    r = requests.get(url)
    data = r.json().get("annualReports", [])
    result = {}
    for report in data:
        year = report.get("fiscalDateEnding", "")[:4]
        if year not in YEARS:
            print(year)
        rd = report.get("researchAndDevelopment", 0)
        rd2 = 0 if rd == 'None' else rd
        revenue = float(report.get("totalRevenue", 0)) * USD_TO_EUR / 1_000_000
        net_income = float(report.get("netIncome", 0)) * USD_TO_EUR / 1_000_000
        rd = float(rd2) * USD_TO_EUR / 1_000_000
        op_cost = revenue - net_income
        result[year] = {
            "Revenue(€M)": round(revenue / 4, 2),
            "Net_Profit(€M)": round(net_income / 4, 2),
            "Operating_Cost(€M)": round(op_cost / 4, 2),
            "R&D_Spending(€M)": round(rd / 4, 2)
        }

    return result

def create_csv(ticker):
    prices = fetch_monthly_prices(ticker)
    financials = fetch_income_statement(ticker)

    with open(f'{ticker}_financial_quarterly_data.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([
            "Quarter", "Revenue(€M)", "Operating_Cost(€M)",
            "Net_Profit(€M)", "R&D_Spending(€M)", "Stock_Price(€M)", "Market_Cap(€B)"
        ])

        for year in YEARS:
            for q, month in QUARTERS.items():
                key = f"{year}-{month}"
                row = [f"{year} {q}"]
                fin = financials.get(year, {})
                row.append(fin.get("Revenue(€M)", ""))
                row.append(fin.get("Operating_Cost(€M)", ""))
                row.append(fin.get("Net_Profit(€M)", ""))
                row.append(fin.get("R&D_Spending(€M)", ""))
                row.append(prices.get(key, ""))
                writer.writerow(row)

    print(f"✅ CSV created: {ticker}_financial_quarterly_data.csv")

create_csv(TICKER)


2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
✅ CSV created: UNH_financial_quarterly_data.csv


In [ ]:
import requests
import pandas as pd

api_key = "R4M2QTIXEG7E3L5E"
tickers = ['ORCL']
for symbol in tickers:
  usd_to_eur = 0.93
  output_csv = f"{symbol}.csv"

  url = f"https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&apikey={api_key}"
  response = requests.get(url)
  data = response.json()

  quarterly_reports = data.get("quarterlyReports", [])
  if not quarterly_reports:
      raise Exception("No quarterly data found. Check your API key or symbol.")

  capex_data = []
  for report in quarterly_reports:
      date = report.get("fiscalDateEnding")
      capex = report.get("capitalExpenditures")

      if date and capex:
          year, month, _ = map(int, date.split("-"))
          if month in [2, 5, 8, 11] and 2017 <= year <= 2024:
              try:
                  capex_million_eur = abs(int(capex)) * usd_to_eur / 1_000_000
                  quarter = f"{year} Q{(month // 3)+1}"
                  capex_data.append({"lol": quarter, "CapEx (€M)": round(capex_million_eur, 2)})
                  print(quarter, round(capex_million_eur, 2))

              except Exception as e:
                  print(f"Error processing data for {date}: {e}")
                  continue
          else:
                  continue

  try:
    df = pd.DataFrame(capex_data)
    df = df.sort_values("lol")
    df.to_csv(output_csv, index=False)
  except Exception as e:
    print(f"Error saving to CSV: {e}")

  print(f"Saved quarterly CapEx data to {output_csv}")

2024 Q4 3692.1
2024 Q3 2141.79
2024 Q2 2602.14
2024 Q1 1556.82
2023 Q4 1004.4
2023 Q3 1222.02
2023 Q2 1779.09
2023 Q1 2444.04
2022 Q4 2264.55
2022 Q3 1598.67
2022 Q2 1323.39
2022 Q1 1023.93
2021 Q4 860.25
2021 Q3 987.66
2021 Q2 666.81
2021 Q1 385.02
2020 Q4 528.24
2020 Q3 405.48
2020 Q2 402.69
2020 Q1 368.28
2019 Q4 324.57
2019 Q3 358.98
2019 Q2 384.09
2019 Q1 411.99
2018 Q4 391.53
2018 Q3 356.19
2018 Q2 351.54
2018 Q1 265.98
2017 Q4 557.07
2017 Q3 439.89
2017 Q2 488.25
2017 Q1 409.2
Saved quarterly CapEx data to ORCL.csv
